<h1>Trabalho de Modelagem e Simulação</h1>
<p style='font-size: 20px'>Aluno: Artur Bernardo Mallmann</p>
Importa classe de

In [30]:
from abc import ABCMeta, abstractmethod, abstractproperty
import sys
import pandas
# class IGen:
#     __metaclass__ = ABCMeta

<h2>Classe para ler CSVs</h2>

In [31]:
from projeto import MyCSVs
arquivo = MyCSVs('entradas.csv')
help(MyCSVs)


# arquivo = pandas.read_csv('entradas.csv')
arquivo.print_html()
# print(arquivo.get_item('ir',0))

Help on class MyCSVs in module projeto:

class MyCSVs(builtins.object)
 |  MyCSVs(file_name)
 |  
 |  Methods defined here:
 |  
 |  __init__(self, file_name)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __iter__(self)
 |  
 |  __next__(self)
 |  
 |  get_col(self, col)
 |  
 |  get_dict(self)
 |  
 |  get_line(self, line, campos=None)
 |  
 |  get_names(self)
 |  
 |  print_html(self)
 |  
 |  where(self, campo, valor, campos=None)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



__start__,fichar,consultar,pagar
3,7,23,4
5,11,15,4
8,8,18,2
9,7,17,3
11,12,22,2
12,10,25,4
14,9,21,3
17,8,22,4
18,10,16,3
21,7,20,2


<h2>Gerar Valores por monte Carlo</h2>
Caso queiramos gerar valores baseado em tempos de chegada amostrados em um CSV, basta passá-lo por por parâmetro

In [32]:
from montecarlo import MonteCarlo
classes = 8 # tipos de amostras diferentes

In [33]:
#mcs é objeto de uma das classes que gera amostras, que tbm pode ser geradores aleatórios quaisquer
mcs = dict()
for en in arquivo.get_names():
    mcs.update({en:MonteCarlo(classes,arquivo.get_col(en))})
# for en in arquivo.keys():
#     mcs.update({en:MonteCarlo(classes,arquivo[en])})

2.25
classes: 8
delta: 2.25
increment: [0, 0, 2, 2, 3, 4, 4, 6, 6, 7]
freq:[0.2, 0.2, 0.4, 0.5, 0.7, 0.7, 0.8999999999999999, 0.9999999999999999]
pms:[4.125, 6.375, 8.625, 10.875, 13.125, 15.375, 17.625, 19.875]
0.625
classes: 8
delta: 0.625
increment: [0, 6, 1, 0, 7, 4, 3, 1, 4, 0]
freq:[0.3, 0.5, 0.5, 0.6, 0.8, 0.8, 0.9, 1.0]
pms:[7.3125, 7.9375, 8.5625, 9.1875, 9.8125, 10.4375, 11.0625, 11.6875]
1.25
classes: 8
delta: 1.25
increment: [6, 0, 2, 1, 5, 7, 4, 5, 0, 4]
freq:[0.2, 0.30000000000000004, 0.4, 0.4, 0.6000000000000001, 0.8, 0.9, 1.0]
pms:[15.625, 16.875, 18.125, 19.375, 20.625, 21.875, 23.125, 24.375]
0.25
classes: 8
delta: 0.25
increment: [7, 7, 0, 4, 0, 7, 4, 7, 4, 0]
freq:[0.3, 0.3, 0.3, 0.3, 0.6, 0.6, 0.6, 1.0]
pms:[2.125, 2.375, 2.625, 2.875, 3.125, 3.375, 3.625, 3.875]


In [34]:
mc = MonteCarlo(10,[1,2,3,10])
# MonteCarlo.__str__ = lambda self:f'{self.get_rand()}'

0.9
classes: 10
delta: 0.9
increment: [0, 1, 2, 9]
freq:[0.25, 0.5, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 1.0]
pms:[1.45, 2.35, 3.25, 4.15, 5.05, 5.95, 6.8500000000000005, 7.75, 8.649999999999999, 9.549999999999999]


In [35]:
print(mcs['__start__'].get_rand())

17.625


In [36]:
print(arquivo.get_names())
# print(arquivo.keys())

['__start__', 'fichar', 'consultar', 'pagar']


<h2>Replay: Ressimular baseado em um CSV</h2>
Caso queira rever uma simulação específica basta passar o CSV por parâmetro.

In [37]:
from projeto import IGen

class Replay(IGen):
    def __init__(self,inputs):
        self.inputs = list(inputs)
    def get_rand(self):
        return self.inputs.pop(0) if len(self.inputs)!=0 else -1 #sempre retira o primeiro

In [38]:
# file_name = 'entradas.csv'
classes = 8
replay = dict()
for en in arquivo.get_names():
    col = arquivo.get_col(en)
    replay.update({en:Replay(arquivo.get_col(en))})
# for en in arquivo.keys():
#     replay.update({en:Replay(arquivo[en])})

In [39]:
print(replay['__start__'].get_rand())

3


<h2>Rand: Entradas randômicas</h2>
Gera os números de entrada de forma randômica.

In [40]:
import random
class GenRand(IGen):
    def __init__(self,mini,maxi):
        self.bounds=(mini,maxi)
    def get_rand(self):
        return random.uniform(*self.bounds)
aaa = GenRand(2,5)
print(aaa.get_rand())

4.646048205997896


<h1>Simulador SimFlex</h1>
O simulador <b>SimFlex</b> tem o intuito de ser uma ferramenta totalmente flexível e configurável pelo usuário.
Os arquivos de configurações descrevem totalmente o funcionamento da simulação. Eles são:
entidades.csv, operacoes.csv e entidades.csv(opcional)

<b>entidades.csv:</b> Arquivo responsável por configurar a alocação de Recursos e dados como tempo de execução em cada processo lógico.

In [70]:
entidades = MyCSVs('entidades.csv')
entidades.print_html()
# entidades = pandas.read_csv('entidades.csv')
# entidades

LP,serv,acoes,tmin,tmax
__src__,50,__start__,2,5
balcao,2,fichar,7,12
balcao,2,pagar,2,4
clinica,3,consultar,15,25
__sync__,0,__stop__,0,0


<b>operacoes.csv:</b> Este arquivo é responsável por descrever qual o fluxo das entidades dentro do sistema

In [71]:
operacoes = MyCSVs('operacoes.csv')
operacoes.print_html()
# operacoes = pandas.read_csv('operacoes.csv')
# operacoes

LP,value,dest,nvalue
__src__,__start__,balcao,fichar
balcao,fichar,clinica,consultar
balcao,pagar,__sync__,__stop__
clinica,consultar,balcao,pagar


<b>entradas.csv</b>(opcional)<b>:</b> Arquivo com a descrição de tempos de eventos pré definidos para gerar novos tempos por monte carlo ou para reproduzir um replay.

In [69]:
entradas = MyCSVs('entradas.csv')
entradas.print_html()
# entradas = pandas.read_csv('entradas.csv')
# entradas

__start__,fichar,consultar,pagar
3,7,23,4
5,11,15,4
8,8,18,2
9,7,17,3
11,12,22,2
12,10,25,4
14,9,21,3
17,8,22,4
18,10,16,3
21,7,20,2


For each funcionando em todos:

In [44]:
for i in entidades:
    print(i)
for i in entradas:
    print (i)
for i in operacoes:
    print (i)
    
operacoes.where('LP','balcao')

('__src__', 50, '__start__', 2, 5)
('balcao', 2, 'fichar', 7, 12)
('balcao', 2, 'pagar', 2, 4)
('clinica', 3, 'consultar', 15, 25)
('__sync__', 0, '__stop__', 0, 0)
(3, 7, 23, 4)
(5, 11, 15, 4)
(8, 8, 18, 2)
(9, 7, 17, 3)
(11, 12, 22, 2)
(12, 10, 25, 4)
(14, 9, 21, 3)
(17, 8, 22, 4)
(18, 10, 16, 3)
(21, 7, 20, 2)
('__src__', '__start__', 'balcao', 'fichar')
('balcao', 'fichar', 'clinica', 'consultar')
('clinica', 'consultar', '__sync__', '__stop__')


[('fichar', 'clinica', 'consultar')]

Mensagems e servidores

In [45]:
class Msg:
    def __init__(self,time,tgt,action):
        self.time=time #relógio do processo
        self.tgt=tgt
        self.action=action # ir,fichar, consultar, pagar, sair
    def run(self,time,tgt,action):
        self.time+=time #relógio do processo
        self.tgt=tgt
        self.action=action # ir,fichar, consultar, pagar, sair
    def log(self):
        return f'(t={self.time},tgt={self.tgt.lpid},action={self.action})'
#         return f'(t={self.time},action={self.action})'
#         return f'(t={self.time},tgt={self.tgt}'
    def __str__(self):
        return f'(t={self.time},tgt={self.tgt.lpid},action={self.action})'
    def __repr__(self):
        return self.__str__()
    
class Spv: #spv service providers (atendentes) ou pode ser usado como fila quando sem entrada.
    def __init__(self, msg, timer):
#         msg.time+=timer.get_rand() # Calcula novo tempo para a mensagem
#         msg.action=
        pass
    def atende(self):
        pass


Processo Lógico, responsável por simular a cadeia de eventos de forma que os eventos desencadeiem na ordem correto, mesmo que assíncrono.

In [46]:
class LogicalProcess:
    def __init__(self,lpid,serv):
        self.lpid = lpid
        self.fila=[] # A princípio filas únicas
        self.serv=serv # entidades simultaneas no LP
        self.timers=dict()
        self.routes=dict()
        self.T = 0 #relógio local
        # Chandy & Misra(1981). Asynchronous distributed simulation via a sequence of parallel computations.
        #
        # All subsequent messages (t, m) sent or received by LPi must have t > Ti. 
        # (i)LPi must have received a message along each input line, (nosso caso a fila que é unica e ordenada)
        # where the t-component of the message is greater than or equal to Ti.
        
        # (ii)LPi must have deduced that it will not send any message (t, m)
        # on any output line where t <= Ti. (LPi can make 
        # this deduction based on its simulation of
        # PPi, it is not necessary for LPi to have actually sent messages (t, m );
        # t >= Ti on every output line provided it can guarantee that
        # t-components of all subsequent output messages will be greater than Ti).
        #
        # A process i computes the clock-value of a line (i, j), which is 
        # a lower bound on the t-component of the next message transmited
        # by i along that line. Process j computes the clock-value of line
        # (i, j) as the t-component of the last message received along (i, j).
        # LP's i and j, in general, will compute different clock-values for
        # line (i, j); it should be clear from the context which value is 
        # meant. Initially clock-value is zero for every line.
        self.ocup = []
        self.acoes = []
    def calc_T(self):
        if(len(self.fila)>0):
            msg = min(self.fila,key=lambda a:a.time)
            self.T = msg.time
        elif(len(self.ocup)>0):
            msg = min(self.ocup,key=lambda a:a.time)
            self.T = msg.time
    def load (self):
        print("%"*50)
        # sort by time
        # tempos de entrada
        self.fila.sort(key=lambda a:a.time,reverse=True) #do menor tempo para o maior
        while len(self.fila)>0 and len(self.ocup)<self.serv: #carregar todos os processos possiveis
            self.calc_T() # calcula o tempo atual
            msg = self.fila.pop() #pega o q chegou por primeiro
            t = self.T + self.timers[msg.action].get_rand() #tempo atual(minimo) + tempo gasto
            nxt,action = self.routes[msg.action]
            msg.run(t,nxt,action) #atribui novos valores à mensagem
            self.ocup.append(msg) #fifo
    def set_timer(self,action,timer):
        self.timers.update({action:timer})
    def qeue(self,msg):
#         if (msg.time >= self.T):
        self.fila.append(msg)
#             return True
#         print(f"recusou {msg}")
#         return False
    def run(self):
        print("%"*50)
#         self.load()
        self.ocup.sort(key=lambda a:a.time,reverse=True)
        remove=[]
#         print(self.ocup)
        
        self.calc_T() # calcula o tempo atual
        for p in self.ocup:
            if(p.time>=self.T):
                resp = p.tgt.qeue(p)
#                 if resp:
                remove+=[p]
        [self.ocup.remove(p) for p in remove]
    def set_route(self,action, destino,naction):
        self.routes.update({action:(destino,naction)})
    def notify(self,msg):
        pass
    
    
    # print

    def showRoutes(self):
        output = "Routes:\n"
        for k in self.routes.keys():
            output += f" {k}:"
            dest,na = self.routes[k]
            output += f" {dest.lpid} - {na}\n"
        return output
    def __str__(self):
        return f'(LPid: {self.lpid} T({self.T})\n fila:[{", ".join([m.log() for m in self.fila])}]\n p-passo:[{", ".join([x.log() for x in self.ocup])}])\n'
    def __repr__(self):
        return self.__str__()
    
# Msg(time,destino,action)

In [63]:
tempo_global=0

class Simulation:
    def __init__ (self,entidades,operacoes,entradas=None,classes=8):
#         tempo_global=0 #Menor tempo global
        
        self.lps=dict()
        # cria os processadores lógicos e já carrega a configuracao de tempo
        for entidade in entidades:
            lpid,serv,action,tmin,tmax = entidade
            lp = None
            try: #se já existir adiciona acoes à classe
                lp = self.lps[lpid]
            except:
                lp = LogicalProcess(lpid,serv)
            
            if(entradas == None or action=="__stop__"):
                lp.set_timer(action,GenRand(tmin,tmax))
            elif classes==0: # Se Classes == 0 irá simular conforme os tempos do arquivo
                lp.set_timer(action,Replay(entradas.get_col(action)))
            else: # Gera tempos por monte carlo
                lp.set_timer(action,MonteCarlo(classes,entradas.get_col(action)))
                    
            self.lps.update({lpid:lp})
        
        for op in operacoes: # Associa a rota dos dados
            lpid, action, dest, naction = op
            self.lps[lpid].set_route(action,self.lps[dest],naction) # conforme value já associa nvalue e destino
#             entidades = 
#             self.acoes[k]=operacoes.where('LP',k)

    def log(self):
        [print(f'LP:{lp}\n{self.lps[lp]}\n{self.lps[lp].showRoutes()}') for lp in self.lps.keys()]
        
# __init__ é o ponto de entrada
    def start(self,n):
                                    # Msg(time,destino,action)
        for i in range(n): # agenda o carregamento de n entidades novas pra dentro da simulação
            self.lps['__src__'].qeue(Msg(0,self.lps['__src__'],'__start__')) # __init__ é responsável por gerar as entidades
        
        self.log()
        i = 0
        while len(self.lps['__sync__'].fila) != n:# and i<100: # começar limitando ciclos
            i+=1
            for lpid in self.lps.keys():
                self.run(lpid)
    def run(self,lpid):
        print(str(i).center(100,'*'))
        print(lpid.center(100,'-'))
        self.lps[lpid].load()
#                 self.log()
        print(self.lps[lpid])
        self.lps[lpid].run() #ciclo em cada LP
        print(self.lps[lpid])
#                 print (f'{k}\n')
#                 self.log()

In [64]:
# sim = Simulation(entidades,operacoes,entradas,10)# monte carlo
# sim = Simulation(entidades,operacoes,entradas,0) # não estocástico (só para depurações)
sim = Simulation(entidades,operacoes) #tempos uniformes
sim.log()

LP:__src__
(LPid: __src__ T(0)
 fila:[]
 p-passo:[])

Routes:
 __start__: balcao - fichar

LP:balcao
(LPid: balcao T(0)
 fila:[]
 p-passo:[])

Routes:
 fichar: clinica - consultar

LP:clinica
(LPid: clinica T(0)
 fila:[]
 p-passo:[])

Routes:
 consultar: __sync__ - __stop__

LP:__sync__
(LPid: __sync__ T(0)
 fila:[]
 p-passo:[])

Routes:



In [65]:
sim.start(20)

LP:__src__
(LPid: __src__ T(0)
 fila:[(t=0,tgt=__src__,action=__start__), (t=0,tgt=__src__,action=__start__), (t=0,tgt=__src__,action=__start__), (t=0,tgt=__src__,action=__start__), (t=0,tgt=__src__,action=__start__), (t=0,tgt=__src__,action=__start__), (t=0,tgt=__src__,action=__start__), (t=0,tgt=__src__,action=__start__), (t=0,tgt=__src__,action=__start__), (t=0,tgt=__src__,action=__start__), (t=0,tgt=__src__,action=__start__), (t=0,tgt=__src__,action=__start__), (t=0,tgt=__src__,action=__start__), (t=0,tgt=__src__,action=__start__), (t=0,tgt=__src__,action=__start__), (t=0,tgt=__src__,action=__start__), (t=0,tgt=__src__,action=__start__), (t=0,tgt=__src__,action=__start__), (t=0,tgt=__src__,action=__start__), (t=0,tgt=__src__,action=__start__)]
 p-passo:[])

Routes:
 __start__: balcao - fichar

LP:balcao
(LPid: balcao T(0)
 fila:[]
 p-passo:[])

Routes:
 fichar: clinica - consultar

LP:clinica
(LPid: clinica T(0)
 fila:[]
 p-passo:[])

Routes:
 consultar: __sync__ - __stop__

LP:__s

Por fim, este é o resultado da simulação:

In [66]:
sim.log()

LP:__src__
(LPid: __src__ T(2.0115903078461703)
 fila:[]
 p-passo:[])

Routes:
 __start__: balcao - fichar

LP:balcao
(LPid: balcao T(19.790620976866165)
 fila:[]
 p-passo:[])

Routes:
 fichar: clinica - consultar

LP:clinica
(LPid: clinica T(59.65702698219373)
 fila:[]
 p-passo:[])

Routes:
 consultar: __sync__ - __stop__

LP:__sync__
(LPid: __sync__ T(44.0316407738911)
 fila:[(t=66.15817192922306,tgt=__sync__,action=__stop__), (t=62.07766458925798,tgt=__sync__,action=__stop__), (t=61.31723524755634,tgt=__sync__,action=__stop__), (t=59.65702698219373,tgt=__sync__,action=__stop__), (t=58.158240800019,tgt=__sync__,action=__stop__), (t=57.20587324042896,tgt=__sync__,action=__stop__), (t=55.79695126655363,tgt=__sync__,action=__stop__), (t=54.36303932548431,tgt=__sync__,action=__stop__), (t=53.91726734401632,tgt=__sync__,action=__stop__), (t=53.41897952451866,tgt=__sync__,action=__stop__), (t=52.586423796604095,tgt=__sync__,action=__stop__), (t=52.106339854355596,tgt=__sync__,action=__stop